In [4]:

import pandas as pd
import numpy as np


In [5]:

bbb_location = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_bbb\\'


In [6]:

match_name_csv = '2023_50_Royal Challengers Bangalore vs Delhi Capitals.csv'


In [7]:

# Load the data into a DataFrame (assuming the data is in a CSV file)
data = pd.read_csv( bbb_location + match_name_csv )


In [12]:

display(data.head(3))


,inn,bat_tm,batnum,bat,ball,over_num,bow,runs,wkt,wkt_how,b.cnt,f.ov.powp,f.ov.midd,f.ov.deat,m.grnd,m.date,ss,m.type,m.num,m.vs
0,1,Royal Challengers Bangalore,1,V Kohli,0,0,KK Ahmed,0,0,NaN,1,1,0,0,Delhi,2023-05-06,2023,T20,50,Royal Challengers Bangalore vs Delhi Capitals
1,1,Royal Challengers Bangalore,1,V Kohli,1,0,KK Ahmed,4,0,NaN,1,1,0,0,Delhi,2023-05-06,2023,T20,50,Royal Challengers Bangalore vs Delhi Capitals
2,1,Royal Challengers Bangalore,1,V Kohli,2,0,KK Ahmed,0,0,NaN,1,1,0,0,Delhi,2023-05-06,2023,T20,50,Royal Challengers Bangalore vs Delhi Capitals


In [8]:

# Extract the unique states: (runs, wickets)
states = sorted(list(set(zip(data['runs'], data['wkt']))))

# Create a mapping of states to indices
state_to_index = {state: index for index, state in enumerate(states)}

# Initialize the transition matrix
transition_matrix = np.zeros((len(states), len(states)))

# Iterate through the data and update the transition matrix
for i in range(len(data) - 1):
    current_state = (data.iloc[i]['runs'], data.iloc[i]['wkt'])
    next_state = (data.iloc[i+1]['runs'], data.iloc[i+1]['wkt'])
    transition_matrix[state_to_index[current_state], state_to_index[next_state]] += 1

# Normalize the transition matrix
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

print("Transition Matrix:")
print(transition_matrix)


Transition Matrix:
[[0.20588235 0.         0.29411765 0.14705882 0.22058824 0.13235294]
 [0.28571429 0.14285714 0.         0.14285714 0.14285714 0.28571429]
 [0.3258427  0.03370787 0.4494382  0.04494382 0.1011236  0.04494382]
 [0.26666667 0.06666667 0.53333333 0.         0.13333333 0.        ]
 [0.41935484 0.06451613 0.4516129  0.         0.03225806 0.03225806]
 [0.29411765 0.         0.41176471 0.         0.17647059 0.11764706]]


In [9]:

def simulate_ball_by_ball(transition_matrix, state_to_index, states, num_balls):
    current_state = (0, 0)
    simulation = []

    for _ in range(num_balls):
        next_state_probs = transition_matrix[state_to_index[current_state]]
        next_state_index = np.random.choice(range(len(states)), p=next_state_probs)
        next_state = states[next_state_index]
        simulation.append(next_state)
        current_state = next_state

    return simulation

# Set the number of balls you want to simulate
num_balls = 120  # For example, a full T20 innings

# Run the simulation
simulated_innings = simulate_ball_by_ball(transition_matrix, state_to_index, states, num_balls)

# Print the simulation results
print("Simulated Innings (Runs, Wickets):")
print(simulated_innings)

# Calculate and print the total runs and wickets
total_runs = sum([ball[0] for ball in simulated_innings])
total_wickets = sum([ball[1] for ball in simulated_innings])
print("Total Runs:", total_runs)
print("Total Wickets:", total_wickets)


Simulated Innings (Runs, Wickets):
[(0, 0), (6, 0), (4, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (1, 0), (6, 0), (1, 0), (0, 0), (0, 0), (2, 0), (1, 0), (1, 0), (0, 0), (6, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (4, 0), (1, 0), (4, 0), (1, 0), (1, 0), (0, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (6, 0), (1, 0), (0, 0), (0, 0), (4, 0), (1, 0), (1, 0), (0, 0), (2, 0), (0, 0), (0, 0), (1, 0), (1, 0), (0, 0), (2, 0), (1, 0), (0, 1), (4, 0), (1, 0), (1, 0), (0, 0), (1, 0), (4, 0), (0, 0), (4, 0), (1, 0), (0, 0), (6, 0), (1, 0), (1, 0), (1, 0), (1, 0), (6, 0), (4, 0), (0, 0), (1, 0), (0, 0), (1, 0), (0, 0), (2, 0), (0, 0), (4, 0), (1, 0), (2, 0), (1, 0), (0, 0), (1, 0), (0, 0), (1, 0), (0, 0), (6, 0), (6, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (6, 0), (1, 0), (4, 0), (1, 0), (0, 0), (2, 0), (4, 0), (0, 0), (1, 0), (6, 0), (1, 0), (1, 0), (1, 0), (0, 1), (2, 0), (1, 0), (0, 0), (0, 0), (4, 0), (0, 0), (4, 0), (0, 0), (0, 0)]
Tota

In [11]:

def create_transition_matrix(data, states, state_to_index):
    transition_matrix = np.zeros((len(states), len(states)))

    for i in range(len(data) - 1):
        current_state = (data.iloc[i]['runs'], data.iloc[i]['wkt'])
        next_state = (data.iloc[i+1]['runs'], data.iloc[i+1]['wkt'])
        transition_matrix[state_to_index[current_state], state_to_index[next_state]] += 1

    transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

    return transition_matrix


In [13]:

first_innings_data = data[data.inn == 1]
second_innings_data = data[data.inn == 2]

first_innings_transition_matrix = create_transition_matrix(first_innings_data, states, state_to_index)
second_innings_transition_matrix = create_transition_matrix(second_innings_data, states, state_to_index)


In [14]:

# Simulate the first innings
simulated_first_innings = simulate_ball_by_ball(first_innings_transition_matrix, state_to_index, states, num_balls)

# Simulate the second innings
simulated_second_innings = simulate_ball_by_ball(second_innings_transition_matrix, state_to_index, states, num_balls)


In [15]:

# Print the simulation results
print("Simulated Innings (Runs, Wickets):")
print(simulated_first_innings)

# Calculate and print the total runs and wickets
total_runs = sum([ball[0] for ball in simulated_first_innings])
total_wickets = sum([ball[1] for ball in simulated_first_innings])
print("Total Runs:", total_runs)
print("Total Wickets:", total_wickets)


Simulated Innings (Runs, Wickets):
[(0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (4, 0), (1, 0), (1, 0), (4, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (2, 0), (1, 0), (0, 0), (1, 0), (4, 0), (1, 0), (0, 0), (4, 0), (1, 0), (1, 0), (1, 0), (6, 0), (0, 0), (2, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 0), (1, 0), (0, 0), (0, 0), (0, 0), (1, 0), (0, 0), (4, 0), (1, 0), (1, 0), (0, 0), (4, 0), (0, 0), (1, 0), (4, 0), (1, 0), (1, 0), (0, 0), (4, 0), (4, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (4, 0), (1, 0), (4, 0), (6, 0), (4, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (2, 0), (1, 0), (0, 0), (1, 0), (1, 0), (2, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (0, 0), (1, 0), (0, 0), (0, 0), (6, 0), (1, 0), (1, 0), (4, 0), (0, 0), (1, 0), (1, 0), (0, 0), (2, 0), (0, 1), (4, 0), (0, 0), (4, 0), (0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Tota

In [16]:

# Print the simulation results
print("Simulated Innings (Runs, Wickets):")
print(simulated_second_innings)

# Calculate and print the total runs and wickets
total_runs = sum([ball[0] for ball in simulated_second_innings])
total_wickets = sum([ball[1] for ball in simulated_second_innings])
print("Total Runs:", total_runs)
print("Total Wickets:", total_wickets)


Simulated Innings (Runs, Wickets):
[(6, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (0, 0), (6, 0), (0, 0), (2, 0), (0, 0), (4, 0), (1, 0), (1, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (1, 0), (6, 0), (1, 0), (0, 0), (2, 0), (4, 0), (1, 0), (1, 0), (0, 0), (1, 0), (4, 0), (0, 1), (2, 0), (0, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (4, 0), (0, 0), (1, 0), (4, 0), (0, 0), (4, 0), (0, 0), (1, 0), (0, 0), (4, 0), (0, 0), (0, 0), (4, 0), (0, 0), (2, 0), (0, 0), (6, 0), (1, 0), (0, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (1, 0), (1, 0), (6, 0), (4, 0), (0, 0), (6, 0), (0, 0), (2, 0), (0, 0), (1, 0), (1, 0), (0, 0), (4, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (4, 0), (0, 0), (2, 0), (1, 0), (1, 0), (1, 0), (1, 0), (4, 0), (1, 0), (1, 0), (0, 1), (0, 0), (4, 0), (0, 0), (0, 0), (1, 0), (0, 0), (6, 0), (1, 0), (0, 0), (6, 0), (0, 0)]
Tota

### CODE FIXED TO CHECK IF THE TARGET HAS BEEN REACHED

In [17]:

def simulate_ball_by_ball(transition_matrix, state_to_index, states, num_balls, target=None):
    current_state = (0, 0)
    simulation = []
    runs = 0
    wickets = 0

    for ball in range(num_balls):
        next_state_probs = transition_matrix[state_to_index[current_state]]
        next_state_index = np.random.choice(range(len(states)), p=next_state_probs)
        next_state = states[next_state_index]
        simulation.append(next_state)
        current_state = next_state
        runs += next_state[0]
        wickets += next_state[1]

        # Stop the simulation if the target is reached or the team has lost all wickets
        if target is not None:
            if runs >= target or wickets == 10:
                break

    return simulation

# Simulate the first innings
simulated_first_innings = simulate_ball_by_ball(first_innings_transition_matrix, state_to_index, states, num_balls)

# Calculate the target for the second team
first_innings_total_runs = sum([ball[0] for ball in simulated_first_innings])
target = first_innings_total_runs + 1

# Simulate the second innings
simulated_second_innings = simulate_ball_by_ball(second_innings_transition_matrix, state_to_index, states, num_balls, target=target)

# Calculate the total runs and wickets for the second innings
second_innings_total_runs = sum([ball[0] for ball in simulated_second_innings])
second_innings_total_wickets = sum([ball[1] for ball in simulated_second_innings])

# Print the results
print("Simulated First Innings (Runs, Wickets):")
print(simulated_first_innings)
print("Total Runs (1st Innings):", first_innings_total_runs)

print("\nSimulated Second Innings (Runs, Wickets):")
print(simulated_second_innings)
print("Total Runs (2nd Innings):", second_innings_total_runs)
print("Total Wickets (2nd Innings):", second_innings_total_wickets)

# Determine the match result
if second_innings_total_runs >= target:
    print("\nSecond team won!")
elif second_innings_total_wickets == 10 or len(simulated_second_innings) == num_balls:
    print("\nSecond team lost!")


Simulated First Innings (Runs, Wickets):
[(4, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (4, 0), (1, 0), (0, 0), (0, 0), (1, 0), (0, 0), (2, 0), (0, 1), (4, 0), (0, 0), (1, 0), (4, 0), (1, 0), (0, 1), (0, 0), (4, 0), (0, 0), (2, 0), (0, 0), (4, 0), (0, 0), (4, 0), (1, 0), (4, 0), (1, 0), (1, 0), (1, 0), (2, 0), (1, 0), (6, 0), (1, 0), (0, 0), (0, 0), (1, 0), (1, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (1, 0), (0, 0), (6, 0), (0, 0), (1, 0), (1, 0), (0, 1), (6, 0), (1, 0), (6, 0), (1, 0), (0, 0), (0, 0), (0, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (0, 0), (0, 0), (2, 0), (1, 0), (6, 0), (1, 0), (0, 0), (2, 0), (1, 0), (2, 0), (1, 0), (4, 0), (1, 0), (2, 0), (1, 0), (1, 0), (6, 0), (0, 0), (2, 0), (1, 0), (2, 0), (1, 0), (0, 0), (0, 0), (6, 0), (1, 0), (6, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 0), (1, 0), (0, 0), (4, 0), (1, 0), (1, 0), (0, 0), (2, 0), (1, 0), (4, 0), (1, 0), (0, 0), (1, 0), (0, 0), (1, 0), (4, 0), (1, 0), (0, 0), (1, 0), (6, 0)

In [18]:

# Function to run a single match simulation
def simulate_match(num_balls):
    simulated_first_innings = simulate_ball_by_ball(first_innings_transition_matrix, state_to_index, states, num_balls)

    first_innings_total_runs = sum([ball[0] for ball in simulated_first_innings])
    target = first_innings_total_runs + 1

    simulated_second_innings = simulate_ball_by_ball(second_innings_transition_matrix, state_to_index, states, num_balls, target=target)

    second_innings_total_runs = sum([ball[0] for ball in simulated_second_innings])
    second_innings_total_wickets = sum([ball[1] for ball in simulated_second_innings])

    if second_innings_total_runs >= target:
        result = "Second team won"
    elif second_innings_total_wickets == 10 or len(simulated_second_innings) == num_balls:
        result = "Second team lost"
    else:
        result = "Match tied"

    return {
        "first_innings_runs": first_innings_total_runs,
        "second_innings_runs": second_innings_total_runs,
        "second_innings_wickets": second_innings_total_wickets,
        "result": result
    }

# Run 100 match simulations and store the results in a list
num_simulations = 100
simulation_results = [simulate_match(num_balls) for _ in range(num_simulations)]

# Print the results
for i, result in enumerate(simulation_results, start=1):
    print(f"Simulation {i}:")
    print(f"  First Innings Runs: {result['first_innings_runs']}")
    print(f"  Second Innings Runs: {result['second_innings_runs']}")
    print(f"  Second Innings Wickets: {result['second_innings_wickets']}")
    print(f"  Result: {result['result']}\n")


Simulation 1:
  First Innings Runs: 148
  Second Innings Runs: 151
  Second Innings Wickets: 2
  Result: Second team won

Simulation 2:
  First Innings Runs: 180
  Second Innings Runs: 186
  Second Innings Wickets: 3
  Result: Second team won

Simulation 3:
  First Innings Runs: 165
  Second Innings Runs: 168
  Second Innings Wickets: 2
  Result: Second team won

Simulation 4:
  First Innings Runs: 155
  Second Innings Runs: 161
  Second Innings Wickets: 5
  Result: Second team won

Simulation 5:
  First Innings Runs: 177
  Second Innings Runs: 178
  Second Innings Wickets: 2
  Result: Second team won

Simulation 6:
  First Innings Runs: 168
  Second Innings Runs: 169
  Second Innings Wickets: 4
  Result: Second team won

Simulation 7:
  First Innings Runs: 156
  Second Innings Runs: 161
  Second Innings Wickets: 1
  Result: Second team won

Simulation 8:
  First Innings Runs: 177
  Second Innings Runs: 178
  Second Innings Wickets: 3
  Result: Second team won

Simulation 9:
  First In

In [19]:

import pandas as pd

# Convert the simulation results to a pandas DataFrame
results_df = pd.DataFrame(simulation_results)

# Calculate the number of matches won by each team
team1_wins = (results_df["result"] == "Second team lost").sum()
team2_wins = (results_df["result"] == "Second team won").sum()

# Calculate the average scores and wickets for each team when they win
team1_wins_df = results_df[results_df["result"] == "Second team lost"]
team2_wins_df = results_df[results_df["result"] == "Second team won"]

team1_avg_score_when_won = team1_wins_df["first_innings_runs"].mean()
team2_avg_score_when_won = team2_wins_df["second_innings_runs"].mean()

team1_avg_wickets_when_won = team2_wins_df["second_innings_wickets"].mean()
team2_avg_wickets_when_won = (10 - team1_wins_df["second_innings_wickets"]).mean()

# Print the summary statistics
print(f"Total matches: {num_simulations}")
print(f"Team 1 wins: {team1_wins} ({(team1_wins/num_simulations) * 100:.2f}%)")
print(f"Team 2 wins: {team2_wins} ({(team2_wins/num_simulations) * 100:.2f}%)")

print("\nAverage scores and wickets when Team 1 wins:")
print(f"  Team 1 average score: {team1_avg_score_when_won:.2f}")
print(f"  Team 2 average score: {team2_avg_score_when_won:.2f}")
print(f"  Team 1 average wickets taken: {team1_avg_wickets_when_won:.2f}")
print(f"  Team 2 average wickets taken: {team2_avg_wickets_when_won:.2f}")

print("\nAverage scores and wickets when Team 2 wins:")
print(f"  Team 1 average score: {team1_avg_score_when_won:.2f}")
print(f"  Team 2 average score: {team2_avg_score_when_won:.2f}")
print(f"  Team 1 average wickets taken: {team1_avg_wickets_when_won:.2f}")
print(f"  Team 2 average wickets taken: {team2_avg_wickets_when_won:.2f}")


Total matches: 100
Team 1 wins: 6 (6.00%)
Team 2 wins: 94 (94.00%)

Average scores and wickets when Team 1 wins:
  Team 1 average score: 192.33
  Team 2 average score: 169.20
  Team 1 average wickets taken: 2.96
  Team 2 average wickets taken: 5.33

Average scores and wickets when Team 2 wins:
  Team 1 average score: 192.33
  Team 2 average score: 169.20
  Team 1 average wickets taken: 2.96
  Team 2 average wickets taken: 5.33
